## **Import and load the data file**

## Test the functionality with the App here: [MedBot](https://www.kaggle.com/discussions/general/497039)

We import the necessary packages for our chatbot and initialize the variables we will use in our Python project.
The data file is in JSON format so we used the json package to parse the JSON file into Python.

In [1]:
import nltk
nltk.download('punkt')      #Sentence tokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [137]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

In [138]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [139]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [140]:
import keras
print(keras.__version__)

2.15.0


# **Preprocessing**

In [141]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open("/content/deda.json").read() # read json file
intents = json.loads(data_file) # load json file

When working with text data, we need to perform various preprocessing on the data before we make a machine learning or a deep learning model. Based on the requirements we need to apply various operations to preprocess the
data.
- Tokenizing is the most basic and first thing you can do on text data.
- Tokenizing is the process of breaking the whole text into small parts like words.
- Here we iterate through the patterns and tokenize the sentence using nltk.word_tokenize() function and append each word in the words list. We also create a list of classes for our tags.

In [142]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)# add each elements into list
        #combination between patterns and intents
        documents.append((w, intent['tag']))#add single element into end of list
        # add to tag in our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [143]:
nltk.download('wordnet') #lexical database for the English language

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [144]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

Now we will lemmatize each word and remove duplicate words from the list.
- Lemmatizing is the process of converting a word into its lemma form and then creating a pickle file to store the Python objects which we will use while predicting.

In [145]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents\n", documents, "\n")
# classes = intents[tag]
print (len(classes), "classes\n", classes, "\n")
# words = all words, vocabulary
print (len(words), "unique lemmatized words\n", words, "\n")
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

294 documents
 [(['Constant', 'fatigue', 'and', 'muscle', 'weakness', ',', 'struggling', 'to', 'stay', 'awake', '.'], 'chronic_fatigue_syndrome'), (['I', "'ve", 'been', 'feeling', 'extremely', 'fatigued', 'and', 'find', 'it', 'hard', 'to', 'concentrate', '.'], 'chronic_fatigue_syndrome'), (['I', 'have', 'been', 'feeling', 'very', 'tired', 'lately', ',', 'even', 'with', 'a', 'full', 'night', "'s", 'sleep', '.'], 'chronic_fatigue_syndrome'), (['Constant', 'fatigue', 'and', 'difficulty', 'concentrating', 'for', 'weeks', '.'], 'chronic_fatigue_syndrome'), (['Extreme', 'fatigue', 'after', 'minimal', 'activity', '.'], 'chronic_fatigue_syndrome'), (['Hi'], 'greeting'), (['Hello'], 'greeting'), (['Hey'], 'greeting'), (['Greetings'], 'greeting'), (['Frequent', 'severe', 'migraines', ',', 'sensitivity', 'to', 'light', 'and', 'sound', '.'], 'migraine'), (['Experiencing', 'frequent', 'headaches', ',', 'sensitivity', 'to', 'light', 'and', 'sound', '.'], 'migraine'), (['Frequent', 'migraines', 'that

# **Training Model**

Now, we will create the training data in which we will provide the input and the output.
- Our input will be the pattern and output will be the class our input pattern belongs to. But the computer doesn’t understand text so we will convert text into numbers

In [146]:
import numpy as np
import random

# Create empty lists for bags and output_rows
bags = []
output_rows = []

# Iterate through documents to create bags and output_rows
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # Create a bag for the current document
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # Create the output row for the current document
    output_row = [0] * len(classes)
    output_row[classes.index(doc[1])] = 1

    # Append the bag and output_row to their respective lists
    bags.append(bag)
    output_rows.append(output_row)

# Convert bags and output_rows to numpy arrays
bags = np.array(bags)
output_rows = np.array(output_rows)

# Combine bags and output_rows into a list of tuples
training = list(zip(bags, output_rows))

# Shuffle training
random.shuffle(training)

# Convert bags and output_rows back to lists for easier indexing
bags, output_rows = zip(*training)

# Convert bags and output_rows to numpy arrays
bags = np.array(bags)
output_rows = np.array(output_rows)

# Split into train and test sets
train_x = bags
train_y = output_rows

# Use the same data for the test for now (you might want to split differently)
test_x = train_x
test_y = train_y

print("Training data created")


Training data created


In [147]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

# **Build the model**

We have our training data ready, now we will build a deep neural network that has 3 layers. We use the Keras sequential API for this. After training the model for 200 epochs, we achieved 100% accuracy on our model. Let us save the model as ‘chatbot_model.h5'.

In [148]:
# Create the model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile the model using Adam optimizer without weight_decay
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [149]:
# # Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# # equal to number of intents to predict output intent with softmax
# model = Sequential()
# model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(len(train_y[0]), activation='softmax'))
# print("First layer:",model.layers[0].get_weights()[0])

In [150]:
# model = Sequential()
# model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(len(train_y[0]), activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [151]:
# Train the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=75, batch_size=5, verbose=1)


Epoch 1/75
59/59 [==============================] - 2s 2ms/step - loss: 4.6258 - accuracy: 0.0068
Epoch 2/75
59/59 [==============================] - 0s 3ms/step - loss: 4.5802 - accuracy: 0.0170
Epoch 3/75
59/59 [==============================] - 0s 3ms/step - loss: 4.5044 - accuracy: 0.0408
Epoch 4/75
59/59 [==============================] - 0s 2ms/step - loss: 4.4398 - accuracy: 0.0612
Epoch 5/75
59/59 [==============================] - 0s 2ms/step - loss: 4.2838 - accuracy: 0.0952
Epoch 6/75
59/59 [==============================] - 0s 2ms/step - loss: 4.1319 - accuracy: 0.1054
Epoch 7/75
59/59 [==============================] - 0s 2ms/step - loss: 3.9029 - accuracy: 0.1565
Epoch 8/75
59/59 [==============================] - 0s 2ms/step - loss: 3.6764 - accuracy: 0.1599
Epoch 9/75
59/59 [==============================] - 0s 2ms/step - loss: 3.5156 - accuracy: 0.1701
Epoch 10/75
59/59 [==============================] - 0s 3ms/step - loss: 3.2900 - accuracy: 0.2449
Epoch 11/75
59/59 [

In [152]:
model.save('medical_chatbot_model.h5')

In [153]:
# # Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
# # sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [154]:
# #fitting and saving the model
# hist = model.fit(np.array(train_x), np.array(train_y), epochs=75, batch_size=5, verbose=1)
# model.save('medical_chatbot_model.h5')

# print("model created")


In [155]:
from keras.models import load_model
model1 = load_model('medical_chatbot_model.h5')

In [156]:
from sklearn.model_selection import train_test_split

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.1, random_state=42)


In [157]:
# Evaluate the model on the test set
loss, accuracy = model1.evaluate(np.array(test_x), np.array(test_y), verbose=1)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

10/10 [==============================] - 0s 2ms/step - loss: 0.0643 - accuracy: 0.9830
Test loss: 0.06432799994945526
Test accuracy: 0.9829931855201721


In [158]:
import tensorflow as tf
import keras
import numpy as np
import nltk

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)
print("NumPy version:", np.__version__)
print("NLTK version:", nltk.__version__)


TensorFlow version: 2.15.0
Keras version: 2.15.0
NumPy version: 1.25.2
NLTK version: 3.8.1


# FOR PREDICTING RESPONSE


In [192]:
intents = json.loads(open(r"/content/deda.json").read())

words = pickle.load(open('words.pkl','rb'))

classes = pickle.load(open('classes.pkl','rb'))

In [193]:
def clean_up_sentence(sentence):

# tokenize the pattern - split words into array

  sentence_words = nltk.word_tokenize(sentence)
#print(sentence_words)
# stem each word - create short form for word

  sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
#print(sentence_words)b

  return sentence_words

In [194]:
def bow(sentence, words, show_details=True):

# tokenize the pattern

  sentence_words = clean_up_sentence(sentence)
  #print(sentence_words)

  # bag of words - matrix of N words, vocabulary matrix

  bag = [0]*len(words)
  #print(bag)

  for s in sentence_words:
      for i,w in enumerate(words):
          if w == s:
              # assign 1 if current word is in the vocabulary position
              bag[i] = 1
              if show_details:
                  print ("found in bag: %s" % w)
              #print ("found in bag: %s" % w)
  #print(bag)
  return(np.array(bag))

In [195]:
def predict_class(sentence, model):

  # filter out predictions below a threshold

  p = bow(sentence, words,show_details=False)
  #print(p)

  res = model.predict(np.array([p]))[0]
  #print(res)

  ERROR_THRESHOLD = 0.25

  results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
  #print(results)
  # sort by strength of probability

  results.sort(key=lambda x: x[1], reverse=True)
  #print(results)

  return_list = []

  for r in results:
      return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

  return return_list
  #print(return_list)

In [196]:
def getResponse(ints, intents_json):

  tag = ints[0]['intent']
  #print(tag)

  list_of_intents = intents_json['intents']
  #print(list_of_intents)

  for i in list_of_intents:
      if(i['tag']== tag):
          result = random.choice(i['responses'])
          break
  return result





In [197]:
def chatbot_response(text):
  ints = predict_class(text, model)

  #print(ints)

  res = getResponse(ints, intents)
  #print(res)
  return res

# Predicting results

i# Text to Text

In [199]:
start = True
while start:

  query = input('Enter Message:')
  if query in ['quit','exit','bye']:
      start = False
      continue
  try:
      res = chatbot_response(query)
      print(res)
  except:
      print('You may need to rephrase your question.')

Enter Message:hi
1/1 [==============================] - 0s 33ms/step
Hello! How can I assist you with Medical Assistance today?
Enter Message:cancer
1/1 [==============================] - 0s 23ms/step
You may need to rephrase your question.
Enter Message:im having fever
1/1 [==============================] - 0s 19ms/step
Stay hydrated and rest. Over-the-counter medications like acetaminophen or ibuprofen can help reduce fever. If the fever is very high or lasts more than a couple of days, consult a doctor.


KeyboardInterrupt: Interrupted by user

# Text to Voice & Text

In [135]:
pip install gtts

In [218]:
from gtts import gTTS
from IPython.display import Audio, display


for _ in range(5):
    query = input('Enter Message: ')
    print("f")
    if query.lower() in ['quit', 'exit', 'bye']:
        start = False
        continue


    res = chatbot_response(query)
    print(res)

    # Convert the response to speech
    tts = gTTS(text=res, lang='en')
    tts.save("response.mp3")
    while True:
      # Play the response without autoplay
      audio = Audio("response.mp3")
      display(audio)
      break

Enter Message: hi
f
1/1 [==============================] - 0s 21ms/step
Hello! How can I assist you with Medical Assistance today?


KeyboardInterrupt: Interrupted by user

In [207]:
import nltk
nltk.download('punkt')  # Sentence tokenizer
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
from gtts import gTTS
from IPython.display import Audio, display

# Your existing code...

start = True
while start:
    query = input('Enter Message:')
    if query.lower() in ['quit', 'exit', 'bye']:
        start = False
        continue

    try:
        res = chatbot_response(query)
        print(res)

        # Convert the response to speech
        tts = gTTS(text=res, lang='en')
        tts.save("response.mp3")

        # Play the response
        display(Audio(filename="response.mp3", autoplay=True))
        continue

    except Exception as e:
        print(f'Error: {e}')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Enter Message:hi
1/1 [==============================] - 0s 21ms/step
Hello! How can I assist you with Medical Assistance today?


KeyboardInterrupt: Interrupted by user

In [166]:
!pip install SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 30.9 MB/s eta 0:00:00


In [167]:
!apt-get install -y libasound-dev portaudio19-dev libportaudio2 libportaudiocpp0


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'libasound2-dev' instead of 'libasound-dev'
libasound2-dev is already the newest version (1.2.6.1-1ubuntu1).
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 0s (604 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 123589 files and directories currently installed.)
Preparin

In [168]:
pip install pyaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyaudio: filename=PyAudio-0.2.14-cp310-cp310-linux_x86_64.whl size=63861 sha256=4f723d38d29c8c6c03dd154a900d8197902eb1aa6fff6a1d20a0787d7be559a7
  Stored in directory: /root/.cache/pip/wheels/d6/21/f4/0b51d41ba79e51b16295cbb096ec49f334792814d545b508c5
Successfully built pyaudio


In [169]:
import speech_recognition as sr

# Print the list of available audio input devices and their indices
print(sr.Microphone.list_microphone_names())


[]


In [170]:
pip install pydub


In [188]:
import nltk
nltk.download('punkt')  # Sentence tokenizer
from nltk.stem import WordNetLemmatizer
import json
import pickle
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
from gtts import gTTS
from IPython.display import Audio, display
from pydub import AudioSegment
from pydub.playback import play
import speech_recognition as sr

# Patch the speech_recognition library to use pydub for audio playback
def patched_play_mp3(audio_file, start=0, end=None):
    audio = AudioSegment.from_mp3(audio_file)
    play(audio[start:end])

sr.AudioFile.play_mp3 = patched_play_mp3

# Your existing code...

def get_voice_input():
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()

    with microphone as source:
        print("Say something:")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source, timeout=5)

    try:
        text = recognizer.recognize_google(audio)
        print("You said:", text)

        # Convert the recognized text to an audio file
        tts = gTTS(text=text, lang='en')
        tts.save("input_audio.mp3")

        # Play the captured audio
        play(AudioSegment.from_mp3("input_audio.mp3"))

        return "input_audio.mp3"

    except sr.UnknownValueError:
        print("Could not understand audio.")
        return None
    except sr.RequestError as e:
        print(f"Error connecting to Google API: {e}")
        return None

# The rest of your code...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [172]:
!pip install pydub sounddevice

In [173]:
!pip install pyttsx3


In [174]:
!pip install ffmpeg-python

In [175]:
!pip install SpeechRecognition pydub

In [176]:
pip install google.colab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.0 MB/s eta 0:00:00


In [178]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
from pydub import AudioSegment
from pydub.playback import play
import time
import io
from pydub import AudioSegment
from pydub.playback import play
import ffmpeg
import numpy as np
from scipy.io.wavfile import read as wav_read






In [179]:
AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    mimeType : 'audio/webm;codecs=opus'
  };
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
    }
  };
  recorder.start();
};

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  resolve(base64data.toString())
});
}
});

</script>
"""

# Voice to Voice

In [219]:

def get_audio():
    display(HTML(AUDIO_HTML))
    data = eval_js("data")

    if not data:
        return None, None

    binary = b64decode(data.split(',')[1])

    # Convert audio data to pydub AudioSegment
    audio_segment = AudioSegment.from_file(io.BytesIO(binary), format="webm")

    # Convert to mono channel
    audio_segment = audio_segment.set_channels(1)

    # Convert to 16-bit PCM format
    audio_segment = audio_segment.set_sample_width(2)

    # Convert to 16 kHz sample rate
    audio_segment = audio_segment.set_frame_rate(16000)

    # Save the processed audio to a file
    audio_segment.export("audio.wav", format="wav")

    # Read the processed audio file
    sr, audio = wav_read("audio.wav")

    return audio, sr

start = True
while start:
    query_audio, _ = get_audio()

    if query_audio is not None:
        try:
            res = chatbot_response(query_audio)
            print(res)

            # Convert the response to speech
            tts = gTTS(text=res, lang='en')
            tts.save("response.mp3")

            # Play the response
            display(Audio(filename="response.mp3", autoplay=True))

        except Exception as e:
            print(f'Error: {e}')

    # Add a delay after processing to prevent continuous recording
    time.sleep(5)  # You can adjust the delay as needed
    start = False  # Stop the recording loop after the first iteration

Error: cannot use a string pattern on a bytes-like object


In [220]:
import speech_recognition as sr
from gtts import gTTS
from IPython.display import Audio, display

def speech_to_text(audio_file):
    recognizer = sr.Recognizer()

    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)

    try:
        text = recognizer.recognize_google(audio)
        return text
    except sr.UnknownValueError:
        print("Could not understand audio.")
        return None
    except sr.RequestError as e:
        print(f"Error connecting to Google API: {e}")
        return None

# Get the text from "audio.wav"
audio_text = speech_to_text("audio.wav")

# Ask the bot using the detected text
if audio_text:
    try:
        res = chatbot_response(audio_text)
        print(res)

        # Convert the response to speech
        tts = gTTS(text=res, lang='en')
        tts.save("response.mp3")

        # Play the response
        display(Audio(filename="response.mp3", autoplay=True))

    except Exception as e:
        print(f'Error: {e}')


Could not understand audio.


In [185]:
pip install pydub numpy scipy sounddevice wavio gtts


In [221]:
from IPython.display import Audio
from base64 import b64decode
from pydub import AudioSegment
from pydub.playback import play
import time
import io
import numpy as np
from scipy.io.wavfile import read as wav_read
import sounddevice as sd
import wavio

def record_audio(duration, sample_rate=16000):
    print("Recording...")
    audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='int16')
    sd.wait()  # Wait until recording is finished
    print("Recording finished.")
    return audio, sample_rate

def save_audio(audio, sample_rate, filename="audio.wav"):
    wavio.write(filename, audio, sample_rate, sampwidth=2)
    return filename

def get_audio():
    # Record audio for 5 seconds
    duration = 5
    audio, sample_rate = record_audio(duration)

    # Save the recorded audio to a file
    audio_file = save_audio(audio, sample_rate)

    # Read the processed audio file
    sr, audio_data = wav_read(audio_file)

    return audio_data, sr

def chatbot_response(audio):
    # Dummy chatbot response function
    # Replace this with your actual chatbot processing logic
    return "This is a test response."

start = True
while start:
    query_audio, sr = get_audio()

    if query_audio is not None:
        try:
            res = chatbot_response(query_audio)
            print(res)

            # Convert the response to speech
            tts = gTTS(text=res, lang='en')
            tts.save("response.mp3")

            # Play the response
            display(Audio(filename="response.mp3", autoplay=True))

        except Exception as e:
            print(f'Error: {e}')

    # Add a delay after processing to prevent continuous recording
    time.sleep(5)  # You can adjust the delay as needed
    start = False  # Stop the recording loop after the first iteration


Recording...


PortAudioError: Error querying device -1